# Module 1 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-01-programming.ipynb
    +--module-01-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID. Make sure you submit both an .ipynb and .html version of your *completed* notebook. You can generate the HTML version using:

> ipython nbconvert [notebookname].ipynb

or use the File menu.

Add any additional standard library imports you need here:

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
.       plains     1
*       forest     3
#       hills      5
~       swamp      7
x       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
....*..
...***.
.###...
..##...
..#..**
....***
.......
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [80]:
full_world = [
  ['.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', '.', '.', '.', '*', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '#', 'x', 'x', '#', '#'], 
  ['.', '.', '.', '.', '#', 'x', 'x', 'x', '*', '*', '*', '*', '~', '~', '*', '*', '*', '*', '*', '.', '.', '#', '#', 'x', 'x', '#', '.'], 
  ['.', '.', '.', '#', '#', 'x', 'x', '*', '*', '.', '.', '~', '~', '~', '~', '*', '*', '*', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.'], 
  ['.', '#', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '~', '~', '~', '~', '~', '.', '.', '.', '.', '.', '#', 'x', '#', '.', '.'], 
  ['.', '#', '#', 'x', 'x', '#', '#', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '.', '.', '.', '#', '.', '.', '.'], 
  ['.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '~', '~', '~', '.', '.', '#', '#', '#', '.', '.'], 
  ['.', '.', '.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '.', '~', '~', '.', '.', '#', '#', '#', '.', '.', '.'], 
  ['.', '.', '.', '~', '~', '~', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '~', '.', '#', '#', '#', '.', '.', '.', '.'], 
  ['.', '.', '~', '~', '~', '~', '~', '.', '#', '#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', 'x', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.', '.', '~', '~', '.', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['~', '~', '~', '~', '~', '.', '.', '#', '#', 'x', 'x', '#', '.', '~', '~', '~', '~', '.', '.', '.', '#', 'x', '#', '.', '.', '.', '.'], 
  ['.', '~', '~', '~', '~', '.', '.', '#', '*', '*', '#', '.', '.', '.', '.', '~', '~', '~', '~', '.', '.', '#', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', '.', '.', '*', '*', '*', '*', '#', '#', '#', '#', '.', '~', '~', '~', '.', '.', '#', 'x', '#', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '#', '#', '.', '~', '.', '#', 'x', 'x', '#', '.', '.', '.'], 
  ['.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', '.', '.', 'x', 'x', 'x', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '*', '*', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.', '.', '.'], 
  ['.', '.', '.', '.', 'x', 'x', 'x', '*', '*', '*', '*', '*', '*', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~'], 
  ['.', '.', '#', '#', '#', '#', 'x', 'x', '*', '*', '*', '*', '*', '.', 'x', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', '*', '*', 'x', 'x', '.', '.', '.', '.', '.', '.', '~', '~', '~', '~', '~', '~', '~'], 
  ['.', '.', '.', '.', '.', '.', '#', '#', '#', 'x', 'x', 'x', 'x', '.', '.', '.', '.', '#', '#', '.', '.', '~', '~', '~', '~', '~', '~'], 
  ['.', '#', '#', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', '#', '#', '.', '~', '~', '~', '~', '~'], 
  ['#', 'x', '#', '#', '#', '#', '.', '.', '.', '.', '.', 'x', 'x', 'x', '#', '#', 'x', 'x', '.', 'x', 'x', '#', '#', '~', '~', '~', '~'], 
  ['#', 'x', 'x', 'x', '#', '.', '.', '.', '.', '.', '#', '#', 'x', 'x', 'x', 'x', '#', '#', '#', '#', 'x', 'x', 'x', '~', '~', '~', '~'], 
  ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '.', '.', '.', '.', '#', '#', '#', '.', '.', '.']]

<div style="background: khaki; color: darkgoldenrod; border: 2px solid goldenrod; padding: 5px; margin: 10px;">
<strong>Warning</strong>
</div>

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7).

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics, what do you expect the policy (plan or path to the goal) would be?

In [81]:
test_world = [
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', 'x', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
]

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.  Moves: the legal movement model expressed in offsets.

In [82]:
cardinal_moves = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [83]:
costs = { '.': 1, '*': 3, '#': 5, '~': 7}

## A\* Search Implementation

As Python is an interpreted language, you're going to need to insert all of your helper functions *before* the actual `a_star_search` function implementation. Put those implementations here, along with their documentation. There should be one Markdown cell for the documentation, followed by one Codecell for the implementation. I've included two to get you started.

-----

### Helper Methods

state[0] = x,y coordinates
state[1] = f(n)
state[2] = cost so far

**Get the cost so far**

To calculate the cost of a state, find its state type by looking at its location in the world.  Use that state type to look up the cost in the costs dictionary.  Add the state's cost to the cost so far of its parent state.

In [ ]:
def get_cost_so_far(state, previous_state, world):
    if previous_state == None:
        previous_cost = 0
    else:
        previous_cost = previous_state[2]
    coordinates = state[0]
    x = coordinates[0]
    y = coordinates[1]
    state_type = world[x][y]
    cost = costs[state_type] + previous_state[2]
    
    return cost
    

-----
**Get f(n)**

Get f(n), which will determine the ordering of our priority queue


In [ ]:
def getFn(heuristic, state, previous_state, world, goal):
    return get_cost_so_far(state, previous_state, world) + heuristic(state, goal)
    

-----
**Insert an item into the frontier list**

The frontier is a priority queue, ordered by f(n).

In [84]:
def insert_into_frontier(heuristic, state, previous_state, world, goal, frontier):
    fn = getFn(heuristic, state, previous_state, world, goal)

    inserted = False
    for i in range len(frontier):
        existing_state = frontier[i]
        existing_fn = state[1]
        
        if fn < existing_fn:
            frontier.inser(i, state)
            inserted = True
    
    if not inserted:
        frontier.append(state)
    
s1 =             
            

-----
**Get the next item from the frontier list**

Get the item with the lowest f(n)

In [85]:
def get_next_state(frontier):
    return frontier[0]

-----
**Determine whether a state is valid**

Determine whether the state is valid given the world****


In [86]:
def is_valid(state, world):
    x = state[0]
    y = state[1]
    
    world_depth = len(world)
    world_width = len(world[0])
    
    if world[x][y] == 'x':
        return False
    
    if x >= 0 and y >= 0 and x < world_depth and y < world_width:
        return True

-----
**Get the next moves available**

Given a state and a world, get the moves based on the given movement model

In [89]:
def get_successors(state, world, moves):
    next_moves = []
        
    for move in moves:
        x = state[0] + move[0]
        y = state[1] + move[1]
        
        new_state = (x,y)
        
        if is_valid(new_state, world):
            next_moves.append(new_state)
        

-----
**Heuristic function**

This heuristic function finds the manhattan distance from the given state to the goal state, assuming a cost of 1.  I assume a cost of 1, to ensure the heuristic is admissible, e.g. does not overestimate the cost to reach the goal.

In [91]:
# heuristic function TODO this goes at the bottom
#Could this be the manhattan distance from the state to the goal???
def heuristic(state, goal):
    xDifference = abs(state[0] - goal[0])
    yDifference = abs(state[1] - goal[1])
    return xDifference + yDifference



-----

**a_star_search**

The `a_star_search` function uses the A\* Search algorithm to solve a navigational problem for an agent in a grid world. It calculates a path from the start state to the goal state and returns the actions required to get from the start to the goal.

* **world** is the starting state representation for a navigation problem.
* **start** is the starting location, `(x, y)`.
* **goal** is the desired end position, `(x, y)`.
* **costs** is a `Dict` of costs for each type of terrain.
* **moves** is the legal movement model expressed in offsets.
* **heuristic** is a heuristic function that returns an estimate of the total cost $f(x)$ from the start to the goal through the current node, $x$. The heuristic function might change with the movement model.


The function returns the offsets needed to get from start state to the goal as a `List`. For example, for the test world:

```
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '*', '*', '*', '*', '*', '*'],
  ['.', '.', '.', '.', '.', '.', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],
  ['*', '*', '*', '*', '*', '*', '.'],

```

it would return:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

Do not make unwarranted assumptions. For example, do not assume the starting point is always `(0, 0)` or that the goal is always in the lower right hand corner. Do not make any assumptions about the movement model beyond the requirement that they be offsets (it could be offets of 2!).

In [68]:
costs2 = { '.': 1, '*': 3, '#': 5, '~': 7}

In [69]:
def a_star_search( world, start, goal, costs, moves, heuristic):
    frontier = []
    explored = []
    path = []
    
    frontier.append(start)

    while frontier:
        current_state = get_next_state(frontier)  
        if current_state == goal:
            return path
        
        children = get_successors(current_state, world, moves)
        for child in children:
            if child not in frontier and not in explored:
                insert_into_frontier(frontier, child)
        
        explored.append(current_state)
    
    return []

a_star_search(test_world, (0,0), (2,2), costs, cardinal_moves, heuristic)

[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[(0, 0)]
[

KeyboardInterrupt: 

**pretty_print_solution**

The `pretty_print_solution` function prints an ASCII representation of the solution generated by the `a_star_search`. For example, for the test world, it would take the `world` and `path` and print:

```
v******
v******
v******
>>>>>>v
******v
******v
******G
```

using `v`, `^`, `>`, `<` to represent actions and `G` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters).


In [ ]:
def pretty_print_solution( world, path, start):
    ### YOUR SOLUTION HERE ###
    ### YOUR SOLUTION HERE ###
    return None

Execute `a_star_search` and `print_path` for the `test_world` and the `real_world`.

*Describe and define your heuristic function here. You can change the arguments to whatever you use in your `a_star_search` implementation.*

In [ ]:
# heuristic function
def heuristic():
    pass

In [ ]:
test_path = a_star_search( test_world, (0, 0), (6, 6), costs, cardinal_moves, heuristic)
print test_path

In [ ]:
pretty_print_solution( test_world, test_path, (0, 0))

In [ ]:
full_path = a_star_search( full_world, (0, 0), (26, 26), costs, cardinal_moves, heuristic)
print full_path

In [ ]:
pretty_print_solution( full_world, full_path, (0, 0))

# Advanced/Future Work

*This section is not required but it is well worth your time to think about the task*

Write a *general* `state_space_search` function that could solve any state space search problem using Depth First Search. One possible implementation would be to write `state_space_search` as a general higher order function that took problem specific functions for `is_goal`, `successors` and `path`. You would need a general way of dealing with states, perhaps as a `Tuple` representing the raw state and metadata: `(<state>, <metadata>)`.